In [2]:
from dotenv import load_dotenv
load_dotenv()

import os
project_name = "wanted_2nd_langchain_outputparser_basic"
os.environ["LANGSMITH_PROJECT"] = project_name

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(
    model_name = "gpt-4.1-mini",
    temperature=0.1,
    verbose=True
)

In [9]:
from typing import Dict
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

In [12]:

prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 AI 도우미야, 간략하게 그냥 응답하도록 해"),
    MessagesPlaceholder(variable_name="history"),
    ("user", "{question}")
])

In [13]:
chain = prompt | model | StrOutputParser()

In [ ]:
store:Dict[str,InMemoryChatMessageHistory] = {}
def get_store(session_id:str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]
        

In [17]:
with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: get_store(session_id),
    input_messages_key="question",
    history_messages_key="history"
)

In [ ]:
cfg = {"configurable":{"session_id":"user-123"}}
result = with_history.invoke({"question":"내가 좋아 하는 폰은 갤럭시야"}, config=cfg)
print(result)

갤럭시 정말 인기 많지! 최신 모델 써봤어?


In [20]:
result = with_history.invoke({"question":"그 장점만 요약해봐"}, config=cfg)
print(result)

갤럭시 장점 요약:
- 뛰어난 디스플레이 품질
- 강력한 성능과 빠른 처리 속도
- 다양한 카메라 기능과 우수한 사진 품질
- 안정적인 소프트웨어 업데이트
- 다양한 모델과 가격대 선택 가능


In [21]:
result = with_history.invoke({"question":"아이폰보다 좋아?"},config=cfg)
print(result)

사용 용도에 따라 다르지만, 갤럭시는 맞춤 설정과 확장성에서 강점이 있어요.


In [22]:
result = with_history.invoke({"question": "너의 개인적인 견해를 말해"}, config=cfg)
print(result)

저는 중립적이라 개인적인 견해는 없어요.


In [23]:
store["user-123"].messages

[HumanMessage(content='내가 좋아 하는 폰은 갤럭시야', additional_kwargs={}, response_metadata={}),
 AIMessage(content='갤럭시 좋은 선택이야! 어떤 모델 좋아해?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='내가 좋아 하는 폰은 갤럭시야', additional_kwargs={}, response_metadata={}),
 AIMessage(content='갤럭시 정말 인기 많지! 최신 모델 써봤어?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='그 장점만 요약해봐', additional_kwargs={}, response_metadata={}),
 AIMessage(content='갤럭시 장점 요약:\n- 뛰어난 디스플레이 품질\n- 강력한 성능과 빠른 처리 속도\n- 다양한 카메라 기능과 우수한 사진 품질\n- 안정적인 소프트웨어 업데이트\n- 다양한 모델과 가격대 선택 가능', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='아이폰보다 좋아?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='사용 용도에 따라 다르지만, 갤럭시는 맞춤 설정과 확장성에서 강점이 있어요.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='너의 개인적인 견해를 말해', additional_kwargs={}, response_metadata={}),
 AIMessage(content='저는 중립적이라 개인적인 견해는 없어요.', additional_kwargs={}, response_metadata={})]